In [1]:
using Pkg
Pkg.activate(".")
using Revise
using Test
using InteractiveUtils
using QuantumToolbox
using QuantumToolbox: check_dimensions, check_mul_dimensions, get_dimensions_from, get_dimensions_to

  Activating project at `/workspaces/QuantumToolbox.jl`

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


In [2]:
QuantumToolbox.versioninfo()



 QuantumToolbox.jl: Quantum Toolbox in Julia
≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡≡
Copyright © QuTiP team 2022 and later.
Current admin team:
    Alberto Mercurio and Yi-Te Huang

Package information:
Julia              Ver. 1.12.1
QuantumToolbox     Ver. nothing
SciMLOperators     Ver. 1.13.0
LinearSolve        Ver. 3.47.0
OrdinaryDiffEqCore Ver. 1.36.0

System information:
OS       : Linux (aarch64-linux-gnu)
CPU      : 14 × unknown
Memory   : 7.653 GB
WORD_SIZE: 64
LIBM     : libopenlibm
LLVM     : libLLVM-18.1.7 (ORCJIT, generic)
BLAS     : libopenblas64_.so (ilp64)
Threads  : 14 (on 14 virtual cores)

+---------------------------------------------------+
| Please cite QuantumToolbox.jl in your publication |
+---------------------------------------------------+
For your convenience, a bibtex reference can be easily generated using `QuantumToolbox.cite()`.



## Cat Definitions

In [4]:
function Base.cat(x::QuantumObject{Ket}...)
    cat(x)
end
function Base.cat(x::Union{AbstractArray{T}, NTuple{N, T}}) where T<:QuantumObject{Ket} where N
    check_dimensions(x)
    to_stack = [x[i].data for i in 1:length(x)]
    stacked_data = hcat(to_stack...)
    new_dims = (x[1].dims, (length(x),))
    return QuantumObject(stacked_data, dims=new_dims)
end

function Base.cat(x :: QuantumObject{Bra}...)
    cat(x)
end

function Base.cat(x :: Union{AbstractArray{T}, NTuple{N, T}}) where T<:QuantumObject{Bra} where N
    check_dimensions(x)
    to_stack = [x[i].data for i in 1:length(x)]
    stacked_data = vcat(to_stack...)
    new_dims = ((length(x),), x[1].dims)
    return QuantumObject(stacked_data, dims=new_dims)
end



function Base.cat(x::QuantumObject{OperatorKet}...)
    cat(x)
end
function Base.cat(x::Union{AbstractArray{T}, NTuple{N, T}}) where T<:QuantumObject{OperatorKet} where N
    check_dimensions(x)
    to_stack = [x[i].data for i in 1:length(x)]
    stacked_data = hcat(to_stack...)
    new_dims = (x[1].dims, (length(x),))
    return QuantumObject(stacked_data, dims=new_dims, type=SuperOperator())
end

function Base.cat(x :: QuantumObject{OperatorBra}...)
    cat(x)
end

function Base.cat(x :: Union{AbstractArray{T}, NTuple{N, T}}) where T<:QuantumObject{OperatorBra} where N
    check_dimensions(x)
    to_stack = [x[i].data for i in 1:length(x)]
    stacked_data = vcat(to_stack...)
    new_dims = ((length(x),), x[1].dims)
    return QuantumObject(stacked_data, dims=new_dims, type=SuperOperator())
end

In [5]:
states = [fock(3,0)⊗fock(2,1), fock(3,1)⊗fock(2,1)];
a = cat(states...)


Quantum Object:   type=Operator()   dims=[[3, 2], [2]]   size=(6, 2)   ishermitian=false
6×2 Matrix{ComplexF64}:
 0.0+0.0im  0.0+0.0im
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im

In [6]:
O = qeye(3)⊗qeye(2);
O*a


Quantum Object:   type=Operator()   dims=[[3, 2], [2]]   size=(6, 2)   ishermitian=false
6×2 Matrix{ComplexF64}:
 0.0+0.0im  0.0+0.0im
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im

In [7]:
check_mul_dimensions(get_dimensions_from(O), get_dimensions_to(a))
O*a


Quantum Object:   type=Operator()   dims=[[3, 2], [2]]   size=(6, 2)   ishermitian=false
6×2 Matrix{ComplexF64}:
 0.0+0.0im  0.0+0.0im
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im

In [8]:
Qobj(rand(4,2), type = SuperOperator())

DomainError: DomainError with (4, 2):
The size of the array is not compatible with SuperOperator

In [9]:
states = [fock(3,0)', fock(3,1)', fock(3,2)'];
b = cat(states...)


Quantum Object:   type=Operator()   dims=[[3], [3]]   size=(3, 3)   ishermitian=true
3×3 Matrix{ComplexF64}:
 1.0-0.0im  0.0-0.0im  0.0-0.0im
 0.0-0.0im  1.0-0.0im  0.0-0.0im
 0.0-0.0im  0.0-0.0im  1.0-0.0im

In [10]:
states = [mat2vec(ket2dm(fock(3,0))), mat2vec(ket2dm(fock(3,1))), mat2vec(ket2dm(fock(3,2)))];
c = cat(states...)

DomainError: DomainError with [[3], [3]]:
The given `dims` is not compatible with type = SuperOperator(), should be a single list of integers.

In [11]:
function check_left_multiplication(A, x)
    A_dims = A.dims[1]
    x_dims = []
    for dim in x.dims[1]
        push!(x_dims, dim)
    end

    println("A_dims: ", A_dims)
    println("x_dims: ", x_dims)
    A_size = size(A)[1]
    x_size = size(x)[1]
    println("A_size: ", A_size)
    println("x_size: ", x_size)
    return A_dims == x_dims && A_size == x_size
end

function check_right_multiplication(A, x)
    A_dims = A.dims[1]
    x_dims = []
    for dim in x.dims[1]
        push!(x_dims, dim)
    end
   
    println("A_dims: ", A_dims)
    println("x_dims: ", x_dims)
    A_size = size(A)[1]
    if length(x.dims) > 1
        x_size = size(x)[2]
    else
        x_size = 1
    end
    println("A_size: ", A_size)
    println("x_size: ", x_size)
    return A_dims == x_dims && A_size == x_size
end

check_right_multiplication (generic function with 1 method)

In [12]:
a = cat(fock(3,0), fock(3,1));
println(a)


Quantum Object:   type=Operator()   dims=[[3], [2]]   size=(3, 2)   ishermitian=false
3×2 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im
 0.0+0.0im  0.0+0.0im


In [13]:
check_left_multiplication(a, fock(3,1)')

A_dims: [3]
x_dims: Any[3]
A_size: 3
x_size: 1


false

In [14]:
check_right_multiplication(a, fock(3,1))

A_dims: [3]
x_dims: Any[3]
A_size: 3
x_size: 1


false

In [15]:
fock(3,1)'*a


Quantum Object:   type=Bra()   dims=[2]   size=(1, 2)
1×2 adjoint(::Vector{ComplexF64}) with eltype ComplexF64:
 0.0-0.0im  1.0-0.0im

In [16]:
size(a')

(2, 3)

## Testing Solver Extensions

### Sesolve

In [ ]:
H = sigmax()
ψ0 = qeye(2) # <- does not work
tlist = 0:0.1:10.0
sol = sesolve(H, ψ0, tlist)

In [ ]:
H = sigmax()
ψ0 = qeye(2) # <- does not work
ψ0 = fock(2,1)
tlist = 0:0.1:10.0
sol = sesolve_map(H, [ψ0,ψ0], tlist)

### ssesolve

I made the changes but i need to look at the documentation for an example

In [5]:
# parameters
N = 4         # Fock space dimension
Δ = 5 * 2 * π  # cavity detuning
κ = 2          # cavity decay rate
α = 4          # intensity of initial state
ntraj = 500    # number of trajectories

tlist = 0:0.0025:1

# operators
a = destroy(N)
x = a + a'
H = Δ * a' * a

# initial state
ψ0 = coherent(N, √α)
ψ0 = ψ0 * ψ0'
# temperature with average of 0 excitations (absolute zero)
n_th = 0
# c_ops  = [√(κ * n_th) * a'] -> nothing
sc_ops = [√(κ * (n_th + 1)) * a]

1-element Vector{QuantumObject{Operator, Dimensions{1, Tuple{Space}}, SparseArrays.SparseMatrixCSC{ComplexF64, Int64}}}:
 
Quantum Object:   type=Operator()   dims=[4]   size=(4, 4)   ishermitian=false
4×4 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 3 stored entries:
     ⋅      1.41421+0.0im      ⋅              ⋅    
     ⋅              ⋅      2.0+0.0im          ⋅    
     ⋅              ⋅          ⋅      2.44949+0.0im
     ⋅              ⋅          ⋅              ⋅    

In [6]:
sse_sol = ssesolve(
    H,
    ψ0,
    tlist,
    sc_ops,
    e_ops=[x],
    ntraj=ntraj,
    store_measurement=Val(true),
)


CompositeException: TaskFailedException

    nested task error: MethodError: no method matching mul!(::SubArray{ComplexF64, 1, Matrix{ComplexF64}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}, ::SparseArrays.SparseMatrixCSC{ComplexF64, Int64}, ::Matrix{ComplexF64}, ::Bool, ::Bool)
    The function `mul!` exists, but no method is defined for this combination of argument types.
    
    Closest candidates are:
      mul!(!Matched::ChainRulesCore.AbstractThunk, ::Any, ::Any, ::Any, ::Any)
       @ ChainRulesCore ~/.julia/packages/ChainRulesCore/Vsbj9/src/tangent_types/thunks.jl:103
      mul!(::AbstractVecOrMat, !Matched::SciMLOperators.IdentityOperator, ::AbstractVecOrMat, ::Any, ::Any)
       @ SciMLOperators ~/.julia/packages/SciMLOperators/gJ83E/src/basic.jl:56
      mul!(::AbstractVecOrMat, !Matched::SciMLOperators.NullOperator, ::AbstractVecOrMat, ::Any, ::Any)
       @ SciMLOperators ~/.julia/packages/SciMLOperators/gJ83E/src/basic.jl:167
      ...
    
    Stacktrace:
      

### mesolve

In [3]:
ψ0 = fock(2)#qeye(4, type = SuperOperator())
ψ0 = qeye(4, type = SuperOperator())
H = sigmaz()
C = 0.001*sigmam()
tlist = 0:0.1:10.0
@inferred mesolve(H, ψ0, tlist, [C])


[mesolve] 100%|███████████████████████████| Time: 0:00:00 (65.90 μs/it)


Solution of time evolution
(return code: Success)
--------------------------
num_states = 101
num_expect = 0
ODE alg.: OrdinaryDiffEqLowOrderRK.DP5{typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!), Static.False}(OrdinaryDiffEqCore.trivial_limiter!, OrdinaryDiffEqCore.trivial_limiter!, static(false))
abstol = 1.0e-8
reltol = 1.0e-6


In [4]:
# ψ0 = fock(2)
# ψ1 = fock(2,1)
H = sigmaz()
C = 0.001 * sigmam()
tlist = 0:0.1:10.0
@inferred mesolve_map(H, [ψ0, ψ0], tlist, [C])

[mesolve_map] 100%|███████████████████████| Time: 0:00:00 (14.98 ms/it)


2-element Vector{TimeEvolutionSol{Vector{Float64}, Vector{Float64}, Vector{QuantumObject{SuperOperator, Dimensions{1, Tuple{Space}}, Matrix{ComplexF64}}}, Nothing, SciMLBase.ReturnCode.T, OrdinaryDiffEqLowOrderRK.DP5{typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!), Static.False}, Float64}}:
 Solution of time evolution
(return code: Success)
--------------------------
num_states = 101
num_expect = 0
ODE alg.: OrdinaryDiffEqLowOrderRK.DP5{typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!), Static.False}(OrdinaryDiffEqCore.trivial_limiter!, OrdinaryDiffEqCore.trivial_limiter!, static(false))
abstol = 1.0e-8
reltol = 1.0e-6

 Solution of time evolution
(return code: Success)
--------------------------
num_states = 101
num_expect = 0
ODE alg.: OrdinaryDiffEqLowOrderRK.DP5{typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!), Static.False}(OrdinaryDiffEqCore.trivial_limiter!, Or

In [12]:
sol[1].states[1]


Quantum Object:   type=SuperOperator()   dims=[2]   size=(4, 4)
4×4 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  1.0+0.0im

### smsolve

In [3]:
# parameters
N = 20         # Fock space dimension
Δ = 5 * 2 * π  # cavity detuning
κ = 2          # cavity decay rate
α = 4          # intensity of initial state
ntraj = 500    # number of trajectories

tlist = 0:0.0025:1

# operators
a = destroy(N)
x = a + a'
H = Δ * a' * a

# initial state
ψ0 = coherent(N, √α)

# temperature with average of 0 excitations (absolute zero)
n_th = 0
# c_ops  = [√(κ * n_th) * a'] -> nothing
sc_ops = [√(κ * (n_th + 1)) * a]

1-element Vector{QuantumObject{Operator, Dimensions{1, Tuple{Space}}, SparseArrays.SparseMatrixCSC{ComplexF64, Int64}}}:
 
Quantum Object:   type=Operator()   dims=[20]   size=(20, 20)   ishermitian=false
20×20 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 19 stored entries:
⎡⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠈⠢⡀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⎦

In [ ]:
sse_sol = ssesolve(
    H,
    ψ0,
    tlist,
    sc_ops,
    e_ops=[x],
    ntraj=ntraj,
    store_measurement=Val(true),
)

measurement_avg = sum(sse_sol.measurement, dims=2) / size(sse_sol.measurement, 2)
measurement_avg = dropdims(measurement_avg, dims=2)


fig

[ssesolve] 100%|██████████████████████████| Time: 0:00:08 (17.56 ms/it)


UndefVarError: UndefVarError: `Figure` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

### mcsolve

In [3]:
tlist = LinRange(0.0, 10.0, 200)

ψ0 = tensor(fock(2, 0), fock(5, 8))
ψ0 = to_dense(qeye(2) ⊗ qeye(5))
a = tensor(qeye(2), destroy(5))
σm = tensor(destroy(2), qeye(5))
H = 2 * π * a' * a + 2 * π * σm' * σm + 2 * π * 0.25 * (σm * a' + σm' * a)

c_ops = [sqrt(0.1) * a]
e_ops = [a' * a, σm' * σm]

sol_500 = mcsolve(H, ψ0, tlist, c_ops, e_ops=e_ops)

[mcsolve] 100%|███████████████████████████| Time: 0:00:00 ( 0.85 ms/it)


Solution of quantum trajectories
(converged: true)
--------------------------------
num_trajectories = 500
num_states = 1
num_expect = 2
ODE alg.: OrdinaryDiffEqLowOrderRK.DP5{typeof(OrdinaryDiffEqCore.trivial_limiter!), typeof(OrdinaryDiffEqCore.trivial_limiter!), Static.False}(OrdinaryDiffEqCore.trivial_limiter!, OrdinaryDiffEqCore.trivial_limiter!, static(false))
abstol = 1.0e-8
reltol = 1.0e-6


In [4]:
sol_500.states[1]


Quantum Object:   type=Operator()   dims=[2, 5]   size=(10, 10)   ishermitian=false
10×10 Matrix{ComplexF64}:
 0.586998+0.0im         0.0+0.0im          …        0.0+0.0im
      0.0+0.0im   -0.457168+1.78283e-9im            0.0+0.0im
      0.0+0.0im         0.0+0.0im                   0.0+0.0im
      0.0+0.0im         0.0+0.0im                   0.0+0.0im
      0.0+0.0im         0.0+0.0im                   0.0+0.0im
      0.0+0.0im  1.55724e-8-0.000909649im  …        0.0+0.0im
      0.0+0.0im         0.0+0.0im                   0.0+0.0im
      0.0+0.0im         0.0+0.0im                   0.0+0.0im
      0.0+0.0im         0.0+0.0im                   0.0+0.0im
      0.0+0.0im         0.0+0.0im             0.0794203-1.18377e-5im